## Trend Observed
1. Charter schools has much higher passing rate than district schools.
2. Schools with higer budget per student have lower average math scores.
3. Smllaer schools have higher average math scores.

constants

In [1]:
import os
from collections import OrderedDict
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

PASSING_SCORE = 60 # Assumption on passing score
passing = lambda x: sum( x >= PASSING_SCORE ) / x.count()
passing.__name__ = 'passing' # To avoid <lambda> in column name

COUNT_FORMAT =  '{:,}'
SCORE_FORMAT = '{:.2f}'
PERC_FORMAT = '{:.1%}'
BUDGET_FORAMT = '${:,.2f}'

OUTPUT_FORMAT = {
    'Total Schools': COUNT_FORMAT,
    'Total Students': COUNT_FORMAT, 
    'Total Budget': BUDGET_FORAMT,
    'Total School Budget': BUDGET_FORAMT,
    'Per Student Budget': BUDGET_FORAMT,
    'Average Math Score': SCORE_FORMAT,
    'Average Reading Score': SCORE_FORMAT,
    '% Passing Math': PERC_FORMAT,
    '% Passing Reading': PERC_FORMAT,
    'Overall Passing Rate (Average of the above two)': PERC_FORMAT,
}

Reading in the source data

In [2]:
BASE_DIR = 'raw_data'
school_file = os.path.join(BASE_DIR, 'schools_complete.csv')
df_school = pd.read_csv(school_file)

student_file = os.path.join(BASE_DIR, 'students_complete.csv')
df_student = pd.read_csv(student_file)

df_school.rename(
    columns={
        'name': 'school'
    },
    inplace=True
)

## District Summary

In [3]:
num_student = df_student.name.count()
passing_rate_math = (df_student.math_score >= PASSING_SCORE).sum() / num_student
passing_rate_reading = (df_student.reading_score > PASSING_SCORE).sum() / num_student
avg_math_score =  df_student.math_score.mean()
avg_reading_score = df_student.reading_score.mean()

district_summary = DataFrame(
    columns = [
        'Total Schools',
        'Total Students',
        'Total Budget',
        'Average Math Score',
        'Average Reading Score',
        '% Passing Math',
        '% Passing Reading',
        'Overall Passing Rate (Average of the above two)',
    ],
    data = [[
        df_school.school.count(),
        num_student,
        df_school.budget.sum(),
        df_student.math_score.mean(),
        df_student.reading_score.mean(),
        passing_rate_math,
        passing_rate_reading,
        (passing_rate_math + passing_rate_reading)/2,
        ]   
    ]
)

district_summary.style.format(OUTPUT_FORMAT)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
0,15,"39,170","$24,649,428.00",78.99,81.88,92.4%,100.0%,96.2%


## School Summary

In [4]:
# Get budget per school
budget_by_school = df_school.groupby(['school', 'type'])['budget'].sum()

# Get summary on student side.
school_summary = df_student.groupby(['school']).agg(
    {
        'name':'count',
        'math_score': ['mean', passing],
        'reading_score': ['mean', passing]
    }
)
# Flatten two level multi-index
school_summary.columns = ['_'.join(col) for col in school_summary.columns]

# Join by index ID ensures values align
school_summary = school_summary.join(budget_by_school).assign(
    per_student_budget=lambda x: x['budget']/x['name_count'],
    average_passing=lambda x: (x['math_score_passing'] + x['reading_score_passing'])/2
).reset_index().rename(
    columns=OrderedDict(
        [
            ('school', 'School Name'),
            ('type','School Type'),
            ('budget', 'Total School Budget'),
            ('name_count', 'Total Students'),
            ('per_student_budget', 'Per Student Budget'),
            ('math_score_mean', 'Average Math Score'),
            ('reading_score_mean', 'Average Reading Score'),
            ('math_score_passing', '% Passing Math'),
            ('reading_score_passing', '% Passing Reading'),
            ('average_passing',  'Overall Passing Rate (Average of the above two)')
        ]
    )
).reindex(
    columns = [
        'School Name',
        'School Type',
        'Total Students',
        'Total School Budget',
        'Per Student Budget',
        'Average Math Score',
        'Average Reading Score',
        '% Passing Math',
        '% Passing Reading',
        'Overall Passing Rate (Average of the above two)'
    ]
)
school_summary.style.format(OUTPUT_FORMAT)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.05,81.03,89.5%,100.0%,94.8%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,100.0%,100.0%,100.0%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,88.4%,100.0%,94.2%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.10,80.75,89.3%,100.0%,94.7%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,100.0%,100.0%,100.0%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,89.1%,100.0%,94.5%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,100.0%,100.0%,100.0%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,88.9%,100.0%,94.4%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,89.2%,100.0%,94.6%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,100.0%,100.0%,100.0%


## Top Perfroming School (By Passing Rate)

In [5]:
school_summary.sort_values('Overall Passing Rate (Average of the above two)',
                    ascending=False).head(5).style.format(OUTPUT_FORMAT)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.06,83.98,100.0%,100.0%,100.0%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.35,83.82,100.0%,100.0%,100.0%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,100.0%,100.0%,100.0%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,100.0%,100.0%,100.0%
11,Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.36,83.73,100.0%,100.0%,100.0%


## Bottom Performing Schools (By Passing Rate)

In [6]:
school_summary.sort_values('Overall Passing Rate (Average of the above two)').head(5).style.format(OUTPUT_FORMAT)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.71,81.16,88.4%,100.0%,94.2%
10,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.84,80.74,88.5%,100.0%,94.3%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.63,81.18,88.9%,100.0%,94.4%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.29,80.93,89.1%,100.0%,94.5%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.07,80.97,89.2%,100.0%,94.6%


## Math/Reading Scores by Grade

In [7]:
df_by_grade = DataFrame(
    df_student.groupby(['school', 'grade'])[['math_score', 'reading_score']].mean()
).rename(
    columns = {
        'school': 'School Name',
        'grade': 'Grade',
        'math_score': 'Average Math Score',
        'reading_score': 'Average Reading Score'
    }
)

df_by_grade.style.format(OUTPUT_FORMAT)

## Scores by School Spending

### Following code logic shared in three reports

In [8]:
def report_scores(df, groupby):
    df_grouped = df.groupby(groupby).agg(
        {
            'math_score': ['mean', passing],
            'reading_score': ['mean', passing]
        }
    )
    
    # Flattern multilevel columns
    df_grouped.columns = ['_'.join(x) for x in df_grouped.columns]
    
    # Computer overal average
    df_grouped = df_grouped.assign(
        overall_rate=lambda x: (x['math_score_passing'] + x['reading_score_passing'] ) / 2
    ).rename(
        columns={
            'math_score_mean': 'Average Math Score',
            'reading_score_mean': 'Average Reading Score',
            'math_score_<lambda>': '% Passing Math',
            'reading_score_<lambda>': '% Passing Reading',
            'overall_rate': 'Overall Passing Rate (Average of the above two)'
        }
    ).reindex(
        columns = [
            'Average Math Score',
            'Average Reading Score',
            '% Passing Math',
            '% Passing Reading',
            'Overall Passing Rate (Average of the above two)'
        ]
    )
    
    return df_grouped.style.format(OUTPUT_FORMAT)

In [9]:
# Computer Bin Size
(min_per_student_budget, max_per_student_budget) = school_summary['Per Student Budget'].agg(['min', 'max'])

# Substract and add 1 to handle float rounding
bins = np.arange(
        min_per_student_budget - 1,
        max_per_student_budget + 1,
         (max_per_student_budget - min_per_student_budget + 1) / 4 
)
labels = ['>={} and <{}'.format(x,y) for x,y in zip(bins, bins[1:])]

school_summary['Budget Range'] = pd.cut(school_summary['Per Student Budget'], bins, labels=labels)

report_scores(
    df_student.merge(
        school_summary, left_on='school', right_on='School Name'
    ),
    'Budget Range'
)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
Budget Range,,,,,
>=577.0 and <596.5,83.36,83.96,nan%,nan%,100.0%
>=596.5 and <616.0,83.53,83.84,nan%,nan%,100.0%
>=616.0 and <635.5,78.48,81.67,nan%,nan%,96.0%
>=635.5 and <655.0,77.42,81.15,nan%,nan%,94.8%


## Scores by School Size

In [10]:
# Computer Bin Size
(min_size, max_size) = df_school['size'].agg(['min', 'max'])

# Substract and add 1 to handle float rounding
bins = np.arange(
        min_size - 1,
        max_size + 1,
        (max_size - min_size + 1) / 3 
)

labels = ['Small', 'Medium', 'Large']

df_school['School Size'] = pd.cut(df_school['size'], bins, labels=labels)

report_scores(
    df_student.merge(
        df_school, on='school'
    ),
    'School Size'
)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
School Size,,,,,
Small,83.44,83.88,nan%,nan%,100.0%
Medium,78.16,81.65,nan%,nan%,95.6%
Large,77.07,80.93,nan%,nan%,94.6%


## Scores by School Type

In [11]:
report_scores(
    df_student.merge(
        df_school, on='school'
    ),
    'type'
)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (Average of the above two)
type,,,,,
Charter,83.41,83.90,nan%,nan%,100.0%
District,76.99,80.96,nan%,nan%,94.5%
